In [ ]:
"""
Code Below are Python code that used to load the data from Source using Boto3 to S3 buckets.
One dataset was very large to and using API to load it was Taking Long time that downloading it
Temporary and delete it after is stored on S3 Buckets.
"""

In [2]:
!pip install boto3

  Obtaining dependency information for botocore<1.32.0,>=1.31.17 from https://files.pythonhosted.org/packages/3d/4c/8f97418ad082458a0d8e6d10434227d54edc6166e3197cee6ecf7b0eeec0/botocore-1.31.85-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 40.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.20.106
    Uninstalling botocore-1.20.106:
      Successfully uninstalled botocore-1.20.106
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.6.0 requires botocore<1.31.18,>=1.31.17, but you have botocore 1.31.85 which is incompatible.


In [3]:
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD

--2023-11-17 01:40:56--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.199, 52.206.140.205, 52.206.68.26
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [       <=>          ]   1.75G  2.95MB/s    in 10m 25s 

2023-11-17 01:51:21 (2.86 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [1873968402]



In [4]:
!mv rows.csv\?accessType\=DOWNLOAD crime_report.csv

In [5]:
import boto3

In [6]:
# Create an S3 client
s3 = boto3.client('s3')

In [7]:
bucket_name = 'final-project-dataset-group6'
local_file_path = 'crime_report.csv'
s3_key = 'crime_report.csv'
try:
    s3.upload_file(local_file_path, bucket_name, s3_key)
    print(f"File {local_file_path} uploaded to S3 as {s3_key}")
except Exception as e:
    print(f"Error uploading file to S3: {str(e)}")


File crime_report.csv uploaded to S3 as crime_report.csv


In [1]:
!rm crime_report.csv

'rm' is not recognized as an internal or external command,
operable program or batch file.


: 

In [ ]:
import pandas as pd
import requests

In [ ]:
# Python function that get dataset in form of json from API
def fetch_data_to_dataframe(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        return df
    else:
        print("Failed to fetch data:", response.status_code)
        return None

In [ ]:
# Extracting Chicago communities area and perimeter
data_url = "https://data.cityofchicago.org/resource/igwz-8jzy.json"
# Fetching data and storing it in a DataFrame (Community areas in Chicago)
chicago_community_data = fetch_data_to_dataframe(data_url)
chicago_community_data.head()

In [ ]:
# Extracting Chicago Police Stations
# URL for the data
data_url = "https://data.cityofchicago.org/resource/z8bn-74gv.json"
# Fetching data and storing it in a DataFrame
chicago_police_stations = fetch_data_to_dataframe(data_url)
chicago_police_stations.head()

In [ ]:
chicago_police_stations['district'].unique()

In [ ]:
from io import StringIO

In [ ]:
# Function That load data to S3 bucket
def load_data_to_s3(df, bucket_name, s3_file_name):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    csv_content = csv_buffer.getvalue()
    try:
        #         s3.upload_file(local_file_path, bucket_name, s3_file_name)
        s3.put_object(Bucket=bucket_name, Key=s3_file_name,
                      Body=csv_content.encode('utf-8'))
        print(
            f"DataFrame uploaded as CSV to S3 bucket: {bucket_name}, Key: {s3_file_name}")
#         print(f"File {local_file_path} uploaded to s3 as {s3_file_name}")
    except Exception as e:
        print(f"Error uploading file to s3: {str(e)}")

In [ ]:
# Calling load dataframe function to load Chicago community areas
load_data_to_s3(chicago_community_data,
                "final-project-dataset-group6", "chicago_communities.csv")

In [ ]:
# calling load data to s3 buckets to police stations dataset
load_data_to_s3(chicago_police_stations,
                "final-project-dataset-group6", "chicago_police_stations.csv")